In [ ]:
def format_vector(state_vector, decimal_precision=7):
    """ Format the state vector into a LaTeX formatted string.

    Args:
        state_vector (list or array): The state vector with complex
                                      values e.g. [-1, 2j+1].

    Returns:
        str: The LaTeX format.
    """
    result = []
    epsilon = 1/pow(10, decimal_precision)
    bit_length = (len(state_vector) - 1).bit_length()
    for index, complex_value in enumerate(state_vector):
        has_imag_part = np.round(complex_value.imag, decimal_precision) != 0.0
        value = complex_value if has_imag_part else complex_value.real
        value_round = np.round(value, decimal_precision)
        if np.abs(value_round) < epsilon:
            continue

        binary_state = '{0:0{1}b}'.format(index, bit_length)
        result.append(r'{0:+2g}\left\lvert {1}\right\rangle '.format(value_round, binary_state))
    return ''.join(result)

In [ ]:
def initialise(q_circuit,q_register,initial_state=0,c_register=None):
    """Initialise quantum circuit `q_circuit` to initial state given as int, 
       which will be converted to binary and applied as initial state (e.g. 6->|110>)
       using Pauli-X gates.
       Returns quantum circuit after initialisation."""
    if c_register: pre_circuit = QuantumCircuit(q_register,c_register)
    else:          pre_circuit = QuantumCircuit(q_register)
    
    state = initial_state
    number_of_qubits = q_register.size
    for kk in range(number_of_qubits):
        if state % 2 == 1:
            pre_circuit.x(q_register[kk])
        state = state // 2
    
    pre_circuit.barrier(q)
    circuit_total = pre_circuit.compose(qc)
    return circuit_total

In [ ]:
def run_circuit(q_circuit,q_register,initial_state='all',c_register=None,shots=1,
                backend=BasicAer.get_backend('statevector_simulator')):
    """Executes circuit gates on register after initialising state. initial_state can be 'all' 
       for which all n-qubit binary states are evaluated, or an int or list/array of ints 
       representing states."""
    number_of_qubits = q_register.size
    
    if initial_state == 'all':
        initial_state = range(2**number_of_qubits)
    elif isinstance(initial_state,int):
        initial_state = [initial_state]
    elif not isinstance(initial_state,(list,np.ndarray)):
        warn(f'initial_state must be an int or list/np.ndarray of ints, not {type(initial_state)}')
    
    results = np.zeros(len(initial_state),dtype=object)
    for i,state in enumerate(initial_state):
        # Initialise circuit
        if c_register: circuit_total = initialise(q_circuit,q_register,state,c_register)
        else:          circuit_total = initialise(q_circuit,q_register,state)
    
        # Circuit execution
        job = execute(circuit_total, backend=backend, shots=shots)
        results[i] = job.result()

        # Output formatting
        try:
            input_state = r'\left\lvert{0:0{1}b}\right\rangle'.format(state, number_of_qubits)
            output_state = job.result().get_statevector(circuit_total)
            latex_text = input_state + r'\mathrm{transforms\ to}: ' + format_vector(output_state)
            display(Math(latex_text))
        except:
            pass
    
    return results,circuit_total

In [ ]:
def state_from_circuit(q_circuit):
    # Circuit execution
    backend=BasicAer.get_backend('statevector_simulator')
    job = execute(q_circuit, backend=backend, shots=1)
    results = job.result()

    # Output formatting
    output_state = job.result().get_statevector(q_circuit)
    display(Math(format_vector(output_state)))

    return output_state